'''
 *--------------------------------------------------------------------------
 *--------------------------------------------------------------------------
 *
 * Copyright (C) 2017 Kareem Abdelfatah - krabea@email.sc.edu
 *
 * The main applications of the StackedGP framework are to integrate different datasets through model composition, 
 * enhance predictions of quantities of interest through a cascade of intermediate predictions, and
 * to propagate uncertainties through emulated dynamical systems driven by uncertain forcing variables. 
 * By using analytical first and second-order moments of a Gaussian process as presented in the 
 * following paper:
 * 
 * Kareem Abdelfatah, Junshu Bao, Gabriel Terejanu (2017). 
 Environmental Modeling Framework using Stacked Gaussian Processes. arXiv:1612.02897v2 . 18 Jun 2017
 * 
 * This program is free software; you can redistribute it and/or modify
 * it under the terms of the GNU General Public License as published by
 * the Free Software Foundation; either version 2 of the License, or
 * (at your option) any later version.
 *
 * This program is distributed in the hope that it will be useful,
 * but WITHOUT ANY WARRANTY; without even the implied warranty of
 * MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
 * GNU General Public License for more details.
 *
 * You should have received a copy of the GNU General Public License along
 * with this program; if not, write to the author.
 *
 *--------------------------------------------------------------------------
 *
 * cadmium_prediction_StackedGP_GP.ipynb
 * 
 *--------------------------------------------------------------------------
 *-------------------------------------------------------------------------- 
 '''

### This File used to predict cadmium using the following stacked structure
    ## First layer: has two nodes
        # input is location
        # output is Ni,Zn
    ## Second layer:
        # input is Zn, Ni, location
        # output is Cd
### The idea here is that you can specify the number of observed Zi, Ni in the prediction phase.
### Also, This file is used to make prediciton using single GP

In [1]:
%matplotlib inline 

import sys
sys.path.append('../')
sys.path.append('../../stackedgp_src/')
from __future__ import division
from mpl_toolkits.mplot3d import axes3d
from stackedGPNetwork import StackedGPNetwork
from sklearn.cross_validation import KFold
import GPy, time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

## Load Data

In [2]:
X = np.genfromtxt('../data/jura2.csv', delimiter=',')[:,[0,1]] # x,y locations
Y = np.genfromtxt('../data/jura2.csv', delimiter=',')[:,[4,10,8]] # Cd, Zn, Ni

print 'Min of Y: ', Y.min(axis=0)
Y[:,[0]] = np.log(Y[:,[0]])
# Y = np.log(Y)
scaler = StandardScaler()
Y[:259,:] = scaler.fit_transform(Y[:259,:])
Y[259:,:] = scaler.transform(Y[259:,:])
# scaler2 = StandardScaler()
# X[:259,2:] = scaler2.fit_transform(X[:259,2:])
# X[259:,2:] = scaler2.transform(X[259:,2:])
print 'normalisation: mean/std',scaler.mean_,scaler.std_

Min of Y:  [  0.135  25.      1.98 ]
normalisation: mean/std [  3.60793618e-02   7.50783012e+01   1.97303475e+01] [  0.70738224  28.96321518   8.2169493 ]


## Train Stacked GP

In [3]:
# ntrain = X.shape[0]
trainX,testX = X[:259,:], X[259:,:]
trainY, testY = Y[:259,0:1], Y[259:,0:1]
trainZ, testZ = Y[:259,1:], Y[259:,1:]
testY = scaler.inverse_transform(np.concatenate((testY,testZ),axis=1))[:,0:1]
testY = np.exp(testY)

In [4]:
stackedNetwork = StackedGPNetwork(2)
stackedNetwork.createNewNode(0,trainX,trainZ[:,0:1], normalize=True, useGPU=False)
stackedNetwork.createNewNode(0,trainX,trainZ[:,1:2], normalize=True, useGPU=False)

traindata = np.concatenate((trainX,trainZ), axis=1)
stackedNetwork.createNewNode(1,traindata,trainY, normalize=True,ARD=False, useGPU=False)


t0= time.clock()
stackedNetwork.optimize(numoptimizationtrails=1)
t= time.clock() - t0 # t is CPU seconds elapsed (floating point)
print 'Training Time = ',t

Training Time =  4.8994


## Test Stacked GP on Testing

In [5]:
#create the input data for the first layer
no_GPs = trainZ.shape[1]
fdata = np.tile(testX,no_GPs)
# predict from the first layer,
fmean, fvar = stackedNetwork.predictLayer(0,fdata,None,include_likelihood=True)

nobservedpoints = 100
sinput = np.concatenate((testZ[:nobservedpoints,:],fmean[nobservedpoints:,:]),axis=0)
svar = np.concatenate((np.zeros(testZ[:nobservedpoints,:].shape),fvar[nobservedpoints:,:]),axis=0)

sdata = np.concatenate((testX,sinput),axis=1)
stv = np.concatenate((np.zeros(testX.shape),svar),axis=1)
mean, var = stackedNetwork.predictLayer(1,sdata,stv, jitter=0, covoption=1, include_likelihood=True,include_covnoise=True)
print 'Applying log-normal distribution inverse...'
E_af = scaler.inverse_transform(np.concatenate((mean,testZ),axis=1))[:,0:1]
E_af = np.exp(E_af)

Applying log-normal distribution inverse...


In [6]:
#=========================================================================
#=========================================================================
# E_af = np.exp(mean+var/2)
# varun = (np.exp(var) -1)*np.exp(2*mean + var)
varun = var
print 'Mean (min/max): ',E_af.min(), E_af.max()
# print 'Variance (min/max): ',varun.min(), varun.max()
# conf1 = E_af + 2*np.sqrt(varun)
# conf2 = E_af - 2*np.sqrt(varun)
rmse = np.sqrt(np.sum(np.square(E_af-testY))/testY.shape[0])
# rmse1,rmse2 =  np.sqrt(np.sum(np.square(conf1-testY))/testY.shape[0]), np.sqrt(np.sum(np.square(conf2-testY))/testY.shape[0])
print 'RMSE: ', rmse
# print '95% conf RMSE: ',rmse1,rmse2
mae = np.sum(abs(E_af-testY))/testY.shape[0]
print 'MAE: ', mae
# mad1,mad2 = np.sum(abs(conf1-testY))/testY.shape[0], np.sum(abs(conf2-testY))/testY.shape[0]
# print '95% conf MAD: ', mad1,mad2
pstd = np.sqrt(varun)
print 'Predicted STD: ', pstd.min(), pstd.max()

Mean (min/max):  0.343353930026 2.26218454622
RMSE:  0.542369066933
MAE:  0.38331762864
Predicted STD:  0.624927897377 0.941767623217


## Train single GP

In [7]:
trainGPX = np.concatenate((trainX,trainZ),axis=1)
testGPX = np.concatenate((testX,testZ),axis=1)
ker = GPy.kern.RBF(trainGPX.shape[1])
m = GPy.models.GPRegression(trainGPX,trainY,ker,normalizer=False)
t2 = time.clock()
m.optimize_restarts(1)
t3= time.clock() - t2 # t is CPU seconds elapsed (floating point)
print 'Training Time = ',t3

Optimization restart 1/1, f = 271.432289953
Training Time =  2.026106


## Test single GP on Testing

In [8]:
gpmean, gpvar = m.predict(testGPX)
#========================================
#=========================================================================
#=========================================================================
#=========================================================================
#=========================================================================
print 'Applying log-normal distribution inverse...'
E_af = scaler.inverse_transform(np.concatenate((gpmean,testZ),axis=1))[:,0:1]
# testY = scaler.inverse_transform(np.concatenate((testY,testZ),axis=1))[:,0:1]
E_af = np.exp(E_af)
# testY = np.exp(testY)
# E_af = np.exp(mean+var/2)
# varun = (np.exp(var) -1)*np.exp(2*mean + var)
varun = var
print 'Mean (min/max): ',E_af.min(), E_af.max()
# print 'Variance (min/max): ',varun.min(), varun.max()
# conf1 = E_af + 2*np.sqrt(varun)
# conf2 = E_af - 2*np.sqrt(varun)
rmse = np.sqrt(np.sum(np.square(E_af-testY))/testY.shape[0])
# rmse1,rmse2 =  np.sqrt(np.sum(np.square(conf1-testY))/testY.shape[0]), np.sqrt(np.sum(np.square(conf2-testY))/testY.shape[0])
print 'RMSE: ', rmse
# print '95% conf RMSE: ',rmse1,rmse2
mae = np.sum(abs(E_af-testY))/testY.shape[0]
print 'MAE: ', mae
# mad1,mad2 = np.sum(abs(conf1-testY))/testY.shape[0], np.sum(abs(conf2-testY))/testY.shape[0]
# print '95% conf MAD: ', mad1,mad2
pstd = np.sqrt(varun)
print 'Predicted STD: ', pstd.min(), pstd.max()

Applying log-normal distribution inverse...
Mean (min/max):  0.343353930844 2.26218454484
RMSE:  0.542369066873
MAE:  0.38331762844
Predicted STD:  0.624927897377 0.941767623217
